In [1]:
import pandas as pd
import numpy as np

# from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
# from sklearn.preprocessing import MinMaxScaler

# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier

c:\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\python37\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\python37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [3]:
# 이상치가 있는 feature 파악

train_outlier = []
features = df_train.columns

for col in features[1:-1]:
    q1_train, q3_train = df_train[col].quantile(.25), df_train[col].quantile(.75)
    iqr_train = q3_train - q1_train
    col_now_train = df_train[col]
    if col_now_train[(q1_train-iqr_train*1.5 > df_train[col]) | (q3_train+iqr_train*1.5 < df_train[col])].count() > 0:
        train_outlier.append(col)

In [4]:
# 결측치를 채우기 전, 결측치 개수 자체를 특징으로 생성한 다음 결측치를 채운다.


df_train_without_nan = df_train.copy()
df_test_without_nan = df_test.copy()

# 결측치 개수 자체를 새로운 특징으로
df_train_without_nan['missing'] = df_train[features[1:-1]].isna().sum(axis=1)
df_test_without_nan['missing'] = df_test[features[1:-1]].isna().sum(axis=1)


# 이상치가 있는 feature 는 median으로,
# 이상치가 없는 feature 는 mean으로
for idx in range(1, 119):
    if f'f{idx}' in train_outlier:
        df_train_without_nan[f'f{idx}'] = df_train_without_nan[f'f{idx}'].fillna(df_train_without_nan[f'f{idx}'].median())
        df_test_without_nan[f'f{idx}'] = df_test_without_nan[f'f{idx}'].fillna(df_test_without_nan[f'f{idx}'].median())
    else:
        df_train_without_nan[f'f{idx}'] = df_train_without_nan[f'f{idx}'].fillna(df_train_without_nan[f'f{idx}'].mean())
        df_test_without_nan[f'f{idx}'] = df_test_without_nan[f'f{idx}'].fillna(df_test_without_nan[f'f{idx}'].mean())
        
        
# 정규화하기 전 분포 정보 자체를 특징으로
df_train_without_nan['std'] = df_train[features[1:-1]].std(axis=1)
df_test_without_nan['std'] = df_test[features[1:-1]].std(axis=1)

In [5]:
# 왜도 skewness 검사 |val| < 1.5

# # 직접 선별
# skew_for_log = ['f3', 'f4', 'f10', 'f21', 'f26', 'f32', 'f33', 'f38', 'f51', 'f53',
#                 'f64', 'f69', 'f74', 'f83', 'f92', 'f96', 'f99', 'f103', 'f104', 'f112',
#                 'f114', 'f115']
# skew_for_sqrt = ['f36', 'f62', 'f73', 'f116']


skew_for_log = ['f3', 'f4', 'f10', 'f21', 'f26', 'f32', 'f33', 'f38', 'f51', 'f53',
                'f64', 'f69', 'f74', 'f83', 'f92', 'f96', 'f99', 'f103', 'f104', 'f112',
                'f114', 'f115']
for col in skew_for_log:
    if min(df_train_without_nan[col]) < 0:
        df_train_without_nan[col] = np.log1p(df_train_without_nan[col] - min(df_train_without_nan[col]))
        df_test_without_nan[col] = np.log1p(df_test_without_nan[col] - min(df_test_without_nan[col]))
    else:
        df_train_without_nan[col] = np.log1p(df_train_without_nan[col])
        df_test_without_nan[col] = np.log1p(df_test_without_nan[col])

In [6]:
features = list(features[1:-1]) + ['missing', 'std'] # 새로 추가한 특성도 정규화가 필요할까?

# scaler = StandardScaler()
scaler = RobustScaler()
# scaler = MinMaxScaler()

# id, claim 제외
df_train_without_nan = scaler.fit_transform(df_train_without_nan[features])
df_test_without_nan = scaler.transform(df_test_without_nan[features])

df_train_without_nan = pd.DataFrame(df_train_without_nan, columns=features)
df_test_without_nan = pd.DataFrame(df_test_without_nan, columns=features)

In [12]:
# SKF = StratifiedKFold(n_splits=5)
SKF = KFold(n_splits=5)

xgbc_oof = np.zeros(df_train_without_nan.shape[0])
xgbc_pred = np.zeros(df_test_without_nan.shape[0])
xgbc_importances = pd.DataFrame()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

for fold, (tr_idx, vld_idx) in enumerate(SKF.split(df_train_without_nan, df_train['claim'])):
    x_tr = df_train_without_nan[features].iloc[tr_idx]
    y_tr = df_train['claim'].iloc[tr_idx]
    
    x_vld = df_train_without_nan[features].iloc[vld_idx]
    y_vld = df_train['claim'].iloc[vld_idx]
    
    XGBC = XGBClassifier(n_jobs=-1,
                         n_estimators=999999,
                         objective='binary:logistic',
                         learning_rate=0.0125,
                         max_depth=9,
                         min_child_weight=91,
                         subsample=0.6094985332043311,
                         colsample_bytree=0.9474073492795716,
                         eval_metric='auc',
                         reg_lambda=0.0219397550092062,
                         reg_alpha=9.313843818151601e-07,
                         early_stopping_rounds=200,
                         random_state=0,
                         tree_method='gpu_hist',
                         predictor='gpu_predictor')
    
    XGBC.fit(x_tr, y_tr,
             eval_set=[(x_vld, y_vld)],
             early_stopping_rounds=200,
             verbose=True)

    xgbc_oof[vld_idx] = XGBC.predict_proba(x_vld)[:, -1]
    xgbc_pred += XGBC.predict_proba(df_test_without_nan)[:, -1] / 5
    
    auc = roc_auc_score(y_vld, xgbc_oof[vld_idx])
    print(f'fold {fold} - CBC auc: {auc:.6f}\n')
    
    feat_importances = pd.DataFrame({'feature_importance': XGBC.feature_importances_}, index=features)
    feat_importances = feat_importances.sort_values(by=['feature_importance'], ascending=False)
    display(feat_importances)

[17:06:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "early_stopping_rounds" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-auc:0.80143
[1]	validation_0-auc:0.80290
[2]	validation_0-auc:0.80285
[3]	validation_0-auc:0.80376
[4]	validation_0-auc:0.80419
[5]	validation_0-auc:0.80470
[6]	validation_0-auc:0.80497
[7]	validation_0-auc:0.80539
[8]	validation_0-auc:0.80557
[9]	validation_0-auc:0.80576
[10]	validation_0-auc:0.80569
[11]	validation_0-auc:0.80582
[12]	validation_0-auc:0.80600
[13]	validation_0-auc:0.80610
[14]	validation_0-auc:0.80627
[15]	validation_0-auc:0.80645
[16]	validation_0-auc:0.80663
[17]	validation_0-auc:0.80682
[18]	validation_0-auc:0.80689
[19]	validation_0-auc:0.80698
[20]	vali

[255]	validation_0-auc:0.81155
[256]	validation_0-auc:0.81154
[257]	validation_0-auc:0.81155
[258]	validation_0-auc:0.81156
[259]	validation_0-auc:0.81158
[260]	validation_0-auc:0.81158
[261]	validation_0-auc:0.81160
[262]	validation_0-auc:0.81163
[263]	validation_0-auc:0.81164
[264]	validation_0-auc:0.81164
[265]	validation_0-auc:0.81165
[266]	validation_0-auc:0.81167
[267]	validation_0-auc:0.81168
[268]	validation_0-auc:0.81170
[269]	validation_0-auc:0.81170
[270]	validation_0-auc:0.81171
[271]	validation_0-auc:0.81173
[272]	validation_0-auc:0.81174
[273]	validation_0-auc:0.81175
[274]	validation_0-auc:0.81176
[275]	validation_0-auc:0.81177
[276]	validation_0-auc:0.81176
[277]	validation_0-auc:0.81177
[278]	validation_0-auc:0.81179
[279]	validation_0-auc:0.81180
[280]	validation_0-auc:0.81180
[281]	validation_0-auc:0.81182
[282]	validation_0-auc:0.81182
[283]	validation_0-auc:0.81185
[284]	validation_0-auc:0.81186
[285]	validation_0-auc:0.81187
[286]	validation_0-auc:0.81186
[287]	va

[520]	validation_0-auc:0.81381
[521]	validation_0-auc:0.81381
[522]	validation_0-auc:0.81381
[523]	validation_0-auc:0.81381
[524]	validation_0-auc:0.81381
[525]	validation_0-auc:0.81382
[526]	validation_0-auc:0.81383
[527]	validation_0-auc:0.81383
[528]	validation_0-auc:0.81384
[529]	validation_0-auc:0.81384
[530]	validation_0-auc:0.81385
[531]	validation_0-auc:0.81385
[532]	validation_0-auc:0.81386
[533]	validation_0-auc:0.81388
[534]	validation_0-auc:0.81390
[535]	validation_0-auc:0.81390
[536]	validation_0-auc:0.81390
[537]	validation_0-auc:0.81391
[538]	validation_0-auc:0.81390
[539]	validation_0-auc:0.81390
[540]	validation_0-auc:0.81392
[541]	validation_0-auc:0.81392
[542]	validation_0-auc:0.81393
[543]	validation_0-auc:0.81392
[544]	validation_0-auc:0.81393
[545]	validation_0-auc:0.81395
[546]	validation_0-auc:0.81396
[547]	validation_0-auc:0.81397
[548]	validation_0-auc:0.81396
[549]	validation_0-auc:0.81398
[550]	validation_0-auc:0.81398
[551]	validation_0-auc:0.81399
[552]	va

[785]	validation_0-auc:0.81487
[786]	validation_0-auc:0.81486
[787]	validation_0-auc:0.81486
[788]	validation_0-auc:0.81486
[789]	validation_0-auc:0.81486
[790]	validation_0-auc:0.81487
[791]	validation_0-auc:0.81487
[792]	validation_0-auc:0.81487
[793]	validation_0-auc:0.81487
[794]	validation_0-auc:0.81487
[795]	validation_0-auc:0.81488
[796]	validation_0-auc:0.81489
[797]	validation_0-auc:0.81490
[798]	validation_0-auc:0.81490
[799]	validation_0-auc:0.81490
[800]	validation_0-auc:0.81490
[801]	validation_0-auc:0.81491
[802]	validation_0-auc:0.81492
[803]	validation_0-auc:0.81492
[804]	validation_0-auc:0.81492
[805]	validation_0-auc:0.81493
[806]	validation_0-auc:0.81493
[807]	validation_0-auc:0.81493
[808]	validation_0-auc:0.81494
[809]	validation_0-auc:0.81494
[810]	validation_0-auc:0.81494
[811]	validation_0-auc:0.81493
[812]	validation_0-auc:0.81493
[813]	validation_0-auc:0.81493
[814]	validation_0-auc:0.81493
[815]	validation_0-auc:0.81493
[816]	validation_0-auc:0.81493
[817]	va

[1048]	validation_0-auc:0.81542
[1049]	validation_0-auc:0.81542
[1050]	validation_0-auc:0.81542
[1051]	validation_0-auc:0.81542
[1052]	validation_0-auc:0.81543
[1053]	validation_0-auc:0.81543
[1054]	validation_0-auc:0.81543
[1055]	validation_0-auc:0.81544
[1056]	validation_0-auc:0.81544
[1057]	validation_0-auc:0.81544
[1058]	validation_0-auc:0.81544
[1059]	validation_0-auc:0.81544
[1060]	validation_0-auc:0.81544
[1061]	validation_0-auc:0.81544
[1062]	validation_0-auc:0.81544
[1063]	validation_0-auc:0.81543
[1064]	validation_0-auc:0.81544
[1065]	validation_0-auc:0.81544
[1066]	validation_0-auc:0.81544
[1067]	validation_0-auc:0.81544
[1068]	validation_0-auc:0.81544
[1069]	validation_0-auc:0.81544
[1070]	validation_0-auc:0.81544
[1071]	validation_0-auc:0.81544
[1072]	validation_0-auc:0.81544
[1073]	validation_0-auc:0.81544
[1074]	validation_0-auc:0.81545
[1075]	validation_0-auc:0.81545
[1076]	validation_0-auc:0.81546
[1077]	validation_0-auc:0.81546
[1078]	validation_0-auc:0.81546
[1079]	v

...

[1819]	validation_0-auc:0.81494
[1820]	validation_0-auc:0.81494
[1821]	validation_0-auc:0.81494
[1822]	validation_0-auc:0.81495
[1823]	validation_0-auc:0.81495
[1824]	validation_0-auc:0.81495
[1825]	validation_0-auc:0.81495
[1826]	validation_0-auc:0.81494
[1827]	validation_0-auc:0.81494
[1828]	validation_0-auc:0.81494
[1829]	validation_0-auc:0.81494
[1830]	validation_0-auc:0.81494
[1831]	validation_0-auc:0.81494
[1832]	validation_0-auc:0.81494
[1833]	validation_0-auc:0.81495
[1834]	validation_0-auc:0.81494
[1835]	validation_0-auc:0.81494
[1836]	validation_0-auc:0.81494
[1837]	validation_0-auc:0.81494
[1838]	validation_0-auc:0.81494
[1839]	validation_0-auc:0.81494
[1840]	validation_0-auc:0.81495
[1841]	validation_0-auc:0.81494
[1842]	validation_0-auc:0.81495
[1843]	validation_0-auc:0.81495
[1844]	validation_0-auc:0.81495
[1845]	validation_0-auc:0.81496
[1846]	validation_0-auc:0.81496
[1847]	validation_0-auc:0.81496
[1848]	validation_0-auc:0.81495
[1849]	validation_0-auc:0.81496
[1850]	v

[2076]	validation_0-auc:0.81516
[2077]	validation_0-auc:0.81516
[2078]	validation_0-auc:0.81516
[2079]	validation_0-auc:0.81516
[2080]	validation_0-auc:0.81516
[2081]	validation_0-auc:0.81516
[2082]	validation_0-auc:0.81516
[2083]	validation_0-auc:0.81516
[2084]	validation_0-auc:0.81517
[2085]	validation_0-auc:0.81517
[2086]	validation_0-auc:0.81517
[2087]	validation_0-auc:0.81517
[2088]	validation_0-auc:0.81517
[2089]	validation_0-auc:0.81518
[2090]	validation_0-auc:0.81518
[2091]	validation_0-auc:0.81518
[2092]	validation_0-auc:0.81518
[2093]	validation_0-auc:0.81518
[2094]	validation_0-auc:0.81518
[2095]	validation_0-auc:0.81518
[2096]	validation_0-auc:0.81518
[2097]	validation_0-auc:0.81518
[2098]	validation_0-auc:0.81518
[2099]	validation_0-auc:0.81518
[2100]	validation_0-auc:0.81518
[2101]	validation_0-auc:0.81518
[2102]	validation_0-auc:0.81518
[2103]	validation_0-auc:0.81518
[2104]	validation_0-auc:0.81518
[2105]	validation_0-auc:0.81518
[2106]	validation_0-auc:0.81518
[2107]	v

[2333]	validation_0-auc:0.81532
[2334]	validation_0-auc:0.81531
[2335]	validation_0-auc:0.81530
[2336]	validation_0-auc:0.81531
[2337]	validation_0-auc:0.81531
[2338]	validation_0-auc:0.81531
[2339]	validation_0-auc:0.81531
[2340]	validation_0-auc:0.81530
[2341]	validation_0-auc:0.81530
[2342]	validation_0-auc:0.81531
[2343]	validation_0-auc:0.81531
[2344]	validation_0-auc:0.81531
[2345]	validation_0-auc:0.81531
[2346]	validation_0-auc:0.81531
[2347]	validation_0-auc:0.81531
[2348]	validation_0-auc:0.81532
[2349]	validation_0-auc:0.81532
[2350]	validation_0-auc:0.81532
[2351]	validation_0-auc:0.81532
[2352]	validation_0-auc:0.81532
[2353]	validation_0-auc:0.81532
[2354]	validation_0-auc:0.81532
[2355]	validation_0-auc:0.81532
[2356]	validation_0-auc:0.81532
[2357]	validation_0-auc:0.81532
[2358]	validation_0-auc:0.81532
[2359]	validation_0-auc:0.81531
[2360]	validation_0-auc:0.81531
[2361]	validation_0-auc:0.81531
[2362]	validation_0-auc:0.81532
[2363]	validation_0-auc:0.81532
[2364]	v

[2590]	validation_0-auc:0.81533
[2591]	validation_0-auc:0.81532
[2592]	validation_0-auc:0.81533
[2593]	validation_0-auc:0.81532
[2594]	validation_0-auc:0.81532
[2595]	validation_0-auc:0.81532
[2596]	validation_0-auc:0.81533
[2597]	validation_0-auc:0.81532
[2598]	validation_0-auc:0.81532
[2599]	validation_0-auc:0.81532
[2600]	validation_0-auc:0.81533
[2601]	validation_0-auc:0.81533
[2602]	validation_0-auc:0.81533
[2603]	validation_0-auc:0.81533
[2604]	validation_0-auc:0.81532
[2605]	validation_0-auc:0.81533
[2606]	validation_0-auc:0.81532
[2607]	validation_0-auc:0.81532
[2608]	validation_0-auc:0.81532
[2609]	validation_0-auc:0.81532
[2610]	validation_0-auc:0.81531
[2611]	validation_0-auc:0.81532
[2612]	validation_0-auc:0.81532
[2613]	validation_0-auc:0.81532
[2614]	validation_0-auc:0.81532
[2615]	validation_0-auc:0.81532
[2616]	validation_0-auc:0.81532
[2617]	validation_0-auc:0.81532
[2618]	validation_0-auc:0.81532
[2619]	validation_0-auc:0.81532
[2620]	validation_0-auc:0.81532
[2621]	v

,feature_importance
missing,0.740860
f40,0.002851
f70,0.002593
f34,0.002572
f47,0.002494
f35,0.002467
f21,0.002462
f45,0.002429
f3,0.002384
f57,0.002382


In [13]:
target = df_train['claim']
print(f'oof cbc roc = {roc_auc_score(target, xgbc_oof)}')

oof cbc roc = 0.8154608608832279


In [14]:
df_submission = pd.read_csv('./sample_solution.csv')
df_submission['claim'] = xgbc_pred
df_submission.to_csv('./xgboost-001.csv', index=False)